In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using PyCall
using JLD2
using LinearAlgebra 
using NearestNeighbors
using Rotations
using GeometryBasics
using GLMakie
using Random

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [97]:
function get_multichromatic_edges_of_tetrahedron(verts; split = 1206)
    a,b,c,d = verts
    combinations = [[a,b], [a,c], [a,d], [b,c], [b,d], [c,d]]
    [e for e in combinations if div(e[1], split) != div(e[2], split)]
end

function get_barycenters_of_multichromatic_edges(edges, points)
    barycenters = Vector{Point3f}([])
    for e in edges
        a,b = e[1],e[2]
        bc = Point3f(0.5 * (points[a+1] + points[b+1]))
        push!(barycenters, bc)
    end
    if length(barycenters) == 4
        bc = Point3f(0.25 * (barycenters[1] + barycenters[2] + barycenters[3] + barycenters[4]))
        push!(barycenters, bc)
    end 
    barycenters
end

function get_faces_from_barycenters(barycenters)
    if length(barycenters) == 3
        faces = [TriangleFace([2, 3, 1])]
    elseif length(barycenters) == 5
        faces = [TriangleFace([2, 4, 5]), TriangleFace([3, 1, 5]), TriangleFace([4, 3, 5]), TriangleFace([1, 2, 5])]
    end
    faces
end

function get_mesh_data(ifil, points)
    bcs = Vector{Point3f}([])
    faces =  Vector{TriangleFace{Int}}([])
    colors = Vector{Float32}([])
    total_vertices = 0
    for (vertices, value) in ifil
        if length(vertices) == 4
            mce = get_multichromatic_edges_of_tetrahedron(vertices)
            t_bcs = get_barycenters_of_multichromatic_edges(mce, points)
            t_faces = get_faces_from_barycenters(t_bcs)
            bcs = [bcs; t_bcs]
            faces = [faces; [f .+ total_vertices for f in t_faces]]
            total_vertices += length(t_bcs)
            colors = [colors; [value for _ in 1:length(t_bcs)]]
        end
    end
    (bcs, faces, colors)
end

get_mesh_data (generic function with 1 method)

In [98]:
@load "../../Data/collected_simulation_results/rwm_wip_2_6r7m/2.jld2" input output

max_v = sqrt(maximum([v for ifil in output["IFILs"] for (_, v) in ifil]))
min_v = sqrt(minimum([v for ifil in output["IFILs"] for (_, v) in ifil]))

realizations = [[Vector{Float64}(e) for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(state, input["template_centers"])...))] for state in output["states"]]
meshdata = [get_mesh_data(ifil, realization) for (ifil, realization) in zip(output["IFILs"], realizations)]
realizations = [hcat(realizations[i]...) for i in 1:length(realizations)]

Es = output["Es"]
mol_type = input["mol_type"]    
exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
thetas = [MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, state) for state in output["states"]]
dgms = output["IDGMs"]
xs = [i for i in 1:length(dgms)];
pw = input["persistence_weights"]
one_ps = [MorphoMol.Energies.get_persistence(dgm[1], pw[1]) for dgm in dgms]
two_ps = [MorphoMol.Energies.get_persistence(dgm[2], pw[2]) for dgm in dgms]
tps = [MorphoMol.Energies.get_total_persistence(dgm, [0.1,0.1]) for dgm in dgms];

f = Figure(fontsize = 12)
#cm = :Accent_8
cm = :Dark2_8
cr = (1,8)

#Slider
sl_i = Slider(f[4, 1:4], range = 1:length(dgms), startvalue = 1)
x = sl_i.value

# Persistence Diagrams
pd_ax_a = Axis(f[1, 1], title = "Persistence codim = 2", xticks = 0:25:100, yticks = 0:25:100)
pd_ax_b = Axis(f[1, 2], title = "Persistence codim = 3", xticks = 0:25:100, yticks = 0:25:100)
dgm_points_2 = @lift([Point2f(dgms[$x][1][i,1], dgms[$x][1][i,2]) for i in 1:size(dgms[$x][1])[1]])
dgm_points_3 = @lift([Point2f(dgms[$x][2][i,1], dgms[$x][2][i,2]) for i in 1:size(dgms[$x][2])[1]])

ms = 5
scatter!(pd_ax_a, dgm_points_2, color = 2, colormap = cm, colorrange = cr, markersize = ms)
lines!(pd_ax_a, [0, 100], [0, 100], color=:black)
scatter!(pd_ax_b, dgm_points_3, color = 3, colormap = cm, colorrange = cr, markersize = ms)
lines!(pd_ax_b, [0, 100], [0, 100], color=:black)

#Energy and Theta
theta_ax = Axis(f[2, 1], title = L"\Theta")
E_ax = Axis(f[2, 2], title = L"F_{sol}")

theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
E_mark = @lift(Point2f($x, $@lift(Es[$x])))

scatter!(theta_ax, xs, thetas, color = 5, colormap = cm, colorrange = cr, markersize = ms)
scatter!(theta_ax, theta_mark, color=:black, markersize = 10)

scatter!(E_ax, xs, Es, color = 6, colormap = cm, colorrange = cr, markersize = ms)
scatter!(E_ax, E_mark, color=:black, markersize = 10)

# Configuration
conf_ax = LScene(f[1:2, 3:4])
n_atoms = size(realizations[1])[2]
n_mol = input["n_mol"]
points = @lift([Point3f(realizations[$x][1,i], realizations[$x][2,i], realizations[$x][3,i]) for i in 1:n_atoms])
colors = vcat([[j for _ in 1:1206] for j in 1:n_mol]...)
conf_ms = 10

scatter!(conf_ax, points, markersize = conf_ms, color = colors, colormap = :rainbow)

# Interface
i_sc = LScene(f[1:2, 5:6])
vertices = [md[1] for md in meshdata]
faces = [md[2] for md in meshdata]
meshes = [GeometryBasics.Mesh(vs, fs) for (vs, fs) in zip(vertices, faces)]

colors = [md[3] for md in meshdata]

l_ms, l_cs = @lift(meshes[$x]), @lift(colors[$x])
mesh!(i_sc, l_ms, color = l_cs, colorrange = (min_v, max_v), colormap = :magma)

#Persistence Measures
one_p_ax = Axis(f[3, 1], title = "$(pw[1]) * total 1 persistence")
two_p_ax = Axis(f[3, 2], title = "$(pw[2]) * total 2 persistence")
tp_ax = Axis(f[3, 3], title = "Total Persistences Summed")

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))

one_p_mark = @lift(Point2f($x, $@lift(one_ps[$x])))
two_p_mark = @lift(Point2f($x, $@lift(two_ps[$x])))

scatter!(one_p_ax, xs, one_ps, color = 2, colormap = cm, colorrange = cr, markersize = ms)
scatter!(one_p_ax, one_p_mark, color=:black, markersize = 10)

scatter!(two_p_ax, xs, two_ps, color = 3, colormap = cm, colorrange = cr, markersize = ms)
scatter!(two_p_ax, two_p_mark, color=:black, markersize = 10)

scatter!(tp_ax, xs, tps, color = 4, colormap = cm, colorrange = cr, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

display(f)

GLMakie.Screen(...)